### 라이브러리

In [1]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from dotenv import load_dotenv

### API 입력

In [2]:
load_dotenv()

# API 키 설정
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY    = os.getenv("OPENAI_API_KEY")

### 문서 로드/분할 및 벡터 임베딩

In [3]:
# PDF 파일 로드 및 분할
loader = PyPDFLoader("KB 실버암 간편건강보험Plus.pdf")
pages = loader.load_and_split()

#PDF 파일을 300자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size    = 200, 
                                               chunk_overlap = 20,
                                               )
docs = text_splitter.split_documents(pages)

#텍스트 임베딩 (OpenAI 사용)
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small",
                              openai_api_key = OPENAI_API_KEY,
                              )

# 벡터 저장소 생성
vectorstore = FAISS.from_documents(docs, 
                                   embedding=embeddings,
                                   )
retriever = vectorstore.as_retriever()

C:\Users\KDT2\AppData\Local\Temp\ipykernel_55108\2623788689.py:12: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model = "text-embedding-3-small",


### 모델 선언

In [4]:
# Claude LLM 선언
llm = ChatAnthropic(model = "claude-opus-4-20250514",
                    temperature = 0,
                    max_tokens = 1024,
                    api_key=ANTHROPIC_API_KEY,
                    )

# RAG 프롬프트 불러오기
prompt = hub.pull("rlm/rag-prompt")

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문 및 답변
question = "통합암진단비ii 는 보장범위가 어떻게 돼?"
answer = rag_chain.invoke(question)
print(answer)


C:\Users\KDT2\AppData\Roaming\Python\Python311\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


통합암진단비Ⅱ는 총 9개의 세부보장으로 구성되어 있으며, 특정소액암진단비Ⅱ, 생식기암 및 비뇨기관암진단비(자궁,전립선,방광제외), 소화기관암진단비, 혈액암진단비 등이 포함됩니다. 암보장개시일 이후 통합암Ⅱ(전이암포함)(유사암제외)로 진단확정된 경우, 암구분에 따라 각각 최초 1회의 진단에 한하여 보험금이 지급됩니다. 각 세부보장별로 보험가입금액의 50% 또는 100%가 지급되는 구조로 되어 있습니다.
